The recommened dataset to work is available here: https://urbansounddataset.weebly.com/urbansound8k.html

In [0]:
import pandas as pd
data = pd.read_csv('...') # Refer to the location of downloaded file here

In [0]:
import librosa
import numpy as np

In [0]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
    return np.hstack([mfccs,chroma,mel,contrast,tonnetz])

In [0]:
from glob import glob
import numpy as np
filenames = np.array(glob('/content/Train/*.wav'))

In [0]:
ids = data['ID'].values

In [0]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    return mfccs

In [0]:
x = []
y = []
for i in range(len(ids)):
  if(i%100==0):
    print(i)
  try:
    filename = '/content/Train/'+str(ids[i])+'.wav'
    y.append(data[data['ID']==ids[i]]['Class'].values)
    x.append(extract_feature(filename))
  except:
    continue
  

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400


In [0]:
np.array(x).shape

(5435, 40)

In [0]:
y2 = []
for i in range(len(y)):
  y2.append(y[i][0])

In [0]:
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
y3 = np.array(pd.get_dummies(y2))

In [0]:
x = np.array(x)

In [0]:
print(x.shape, y3.shape)

(5435, 40) (5435, 10)


In [0]:
x = x.reshape(x.shape[0],x.shape[1],1)

In [0]:
x = x.reshape(x.shape[0],x.shape[1])

In [0]:
import os
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import Conv2D, ZeroPadding2D, BatchNormalization, Input, Dropout
from keras.layers import Conv2DTranspose, Reshape, Activation, Cropping2D, Flatten
from keras.layers import Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import relu
from keras.initializers import RandomNormal
from keras.layers import Embedding
from keras.layers import LSTM, RepeatVector, TimeDistributed, Dense, Bidirectional

In [0]:
model = Sequential()
model.add(Dense(1000, input_shape = (40,), activation = 'relu'))
#model.add(Dense(1000,activation='relu'))
model.add(Dense(10,activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              41000     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                10010     
Total params: 51,010
Trainable params: 51,010
Non-trainable params: 0
_________________________________________________________________


In [0]:
model = Sequential()
model.add(Dense(1000, input_shape = (193,1), activation = 'relu'))
model.add((LSTM(100)))
model.add(Dense(1000,activation='relu'))
model.add(Dense(10,activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 193, 1000)         2000      
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               440400    
_________________________________________________________________
dense_14 (Dense)             (None, 1000)              101000    
_________________________________________________________________
dense_15 (Dense)             (None, 10)                10010     
Total params: 553,410
Trainable params: 553,410
Non-trainable params: 0
_________________________________________________________________


In [0]:
from keras.optimizers import Adam
adam = Adam(lr=0.0001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])

In [0]:
np.max(x)

263.54005695096265

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y3, test_size=0.30,random_state=10)

In [0]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose = 1)

Train on 3804 samples, validate on 1631 samples
Epoch 1/100
3804/3804 [==============================] - 3s 750us/step - loss: 2.6400 - acc: 0.1207 - val_loss: 2.0312 - val_acc: 0.1337
Epoch 2/100
3804/3804 [==============================] - 1s 155us/step - loss: 1.9418 - acc: 0.1296 - val_loss: 1.8246 - val_acc: 0.1527
Epoch 3/100
3804/3804 [==============================] - 1s 158us/step - loss: 1.7654 - acc: 0.1562 - val_loss: 1.6738 - val_acc: 0.1717
Epoch 4/100
3804/3804 [==============================] - 1s 166us/step - loss: 1.5467 - acc: 0.2581 - val_loss: 1.3530 - val_acc: 0.4316
Epoch 5/100
3804/3804 [==============================] - 1s 166us/step - loss: 1.2187 - acc: 0.5602 - val_loss: 1.0379 - val_acc: 0.6383
Epoch 6/100
3804/3804 [==============================] - 1s 168us/step - loss: 1.0041 - acc: 0.6485 - val_loss: 0.9501 - val_acc: 0.6824
Epoch 7/100
3804/3804 [==============================] - 1s 165us/step - loss: 0.8735 - acc: 0.7095 - val_loss: 0.8983 - val_acc: 